In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
class BasicLightningTrain(L.LightningModule):
    def __init__ (self):
        super().__init__()

        self.w00 = nn.Parameter(torch.tensor(1.7), requires_grad=False)
        self.b00 = nn.Parameter(torch.tensor(-0.85), requires_grad=False)
        self.w01 = nn.Parameter(torch.tensor(-40.8), requires_grad=False)

        self.w10 = nn.Parameter(torch.tensor(12.6), requires_grad=False)
        self.b10 = nn.Parameter(torch.tensor(0.0), requires_grad=False)
        self.w11 = nn.Parameter(torch.tensor(2.7), requires_grad=False)

        self.final_bias = nn.Parameter(torch.tensor(0.0), requires_grad=True)
        self.learning_rate = 0.01

    def forward(self, input):
        input_to_top_relu = self.w00 * input + self.b00
        top_relu_output = F.relu(input_to_top_relu)
        scaled_top_relu_output = self.w01 * top_relu_output

        input_to_bottom_relu = self.w10 * input + self.b10
        bottom_relu_output = F.relu(input_to_bottom_relu)
        scaled_bottom_relu_output = self.w11 * bottom_relu_output

        input_to_final_relu = self.final_bias + scaled_bottom_relu_output + scaled_top_relu_output

        output = F.relu(input_to_final_relu)
        return output
    
    def configure_optimizers(self):
        # self.parameters() returns the values that can be trained, i.e. where the requires_grad = True
        # basically we are returning -------> "Use SGD to tweak final_bias with a step size of 0.01 to minimize the loss."
        return SGD(self.parameters(), lr=self.learning_rate)
    
    # batch -> chunk of data provided by the DataLoader
    # input_i, output_i will be assigned the value which is given to the batch 
    def training_step(self, batch, batch_idx):
        # What is a batch, anyway?
        # Think of it like a small group of examples the model learns from at once
        input_i, label_i = batch
        # taking the input and doing the math using forward function and storing the output for that input in the output_i 
        output_i = self.forward(input_i)
        # predicted_output - actual_output ** 2
        loss = (output_i - label_i) ** 2
        return loss

In [9]:
inputs = torch.tensor([0., 0.5, 1.0])
labels = torch.tensor([0.0, 1.0, 0.0])

dataset = TensorDataset(inputs, labels)

# this is what a batch uses in the training_step function
# dataloader basically have the whole pair of inputs, labels, i.e dataset, and it would serve one batch at a time to the model for training
dataloader = DataLoader(dataset)

In [ ]:
model = BasicLightningTrain()

# Trainer --> responsible for running the training, testing, or tuning of your model. It handles all the complicated stuff—like looping over data, calling the optimizer, and tracking progress
trainer = L.Trainer(max_epochs=100)

tuner = L.pytorch.tuner.Tuner(trainer)

lr_finder = tuner.lr_find(model, train_dataloaders = dataloader, min_lr = 0.01, max_lr = 1.0, early_stop_threshold=None)

lr_result = lr_finder.suggestion()
print(lr_result)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Finding best initial lr: 100%|██████████| 100/100 [00:00<00:00, 346.25it/s]
Learning rate set to 0.016595869074375606
Restoring states from the checkpoint path at c:\Users\11ukn\OneDrive\Desktop\Project ASUNA\Projects (ML)\Deep Learning - Duality\.lr_find_a57e926c-ee32-4932-87c2-8ab4f7aa1ac4.ckpt
Restored all states from the checkpoint at c:\Users\11ukn\OneDrive\Desktop\Project ASUNA\Projects (ML)\Deep Learning - Duality\.lr_find_a57e926c-ee32-4932-87c2-8ab4f7aa1ac4.ckpt


0.016595869074375606


In [17]:
model.learning_rate = lr_result
trainer.fit(model, train_dataloaders=dataloader)


  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 7      | n/a 
---------------------------------------------
1         Trainable params
6         Non-trainable params
7         Total params
0.000     Total estimated model params size (MB)
0         Modules in train mode
0         Modules in eval mode


Epoch 99: 100%|██████████| 3/3 [00:00<00:00, 204.86it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 3/3 [00:00<00:00, 129.52it/s, v_num=2]


In [18]:
print(model.final_bias.data)

tensor(-15.4624)
